In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from matplotlib import colormaps
import cv2
import os

%config InlineBackend.figure_format='svg'
%matplotlib inline
plt.style.use('ggplot')

cmap = colormaps["gray"]
width = 28

In [2]:
x, y = fetch_openml("cifar_10", return_X_y=True, as_frame=False)

In [3]:
x = x / 255.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

print(list(map(len, [x_train, x_test, y_train, y_test])))

[45000, 15000, 45000, 15000]


In [5]:
K = 3

id_part = np.random.randint(0, len(x_train), 5000)

knc = KNeighborsClassifier(n_neighbors=K)
knc = knc.fit(x_train[id_part], y_train[id_part])

y_predicted = knc.predict(x_test)
score = accuracy_score(y_test, y_predicted)

print(f"{K = }")
print(f"{score = }")

K = 3
score = 0.25326666666666664


In [6]:
id_part = np.random.randint(0, len(x_train), 2000)

svc = SVC()
svc = svc.fit(x_train[id_part], y_train[id_part])

y_predicted = svc.predict(x_test)
score = accuracy_score(y_test, y_predicted)

print(f"{score = }")

score = 0.3946


https://ermlab.com/en/blog/nlp/cifar-10-classification-using-keras-tutorial/

In [32]:
from keras.layers import Dense, Dropout
from keras import Sequential
from keras.utils import to_categorical


model = Sequential()

model.add(Dense(128, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))
    
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer='sgd', loss='categorical_crossentropy')

y_train_c = to_categorical(y_train)
model.fit(x_train, y_train_c, batch_size=32, epochs=10)

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_64 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.0495
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.7699
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.6632
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.6040
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.5413
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.5109
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.4701
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.4426
Epoch 9/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.4222
Epoch 10/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.3839


In [45]:
pred = model.predict(x_test)
y_pred = list(map(str, np.argmax(pred, axis=-1)))

score = accuracy_score(y_test, y_pred)
print(f"{score = }")

469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
score = 0.42473333333333335


In [58]:
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras import Sequential
from keras.utils import to_categorical

model1 = Sequential()
model1.add(Conv2D(32, (3,3), activation='relu'))

model1.add(Dense(128, activation='relu'))
# model1.add(MaxPooling2D(pool_size=(2,2)))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(64, activation='relu'))
    
model1.add(Dense(10, activation='softmax'))

model1.compile(optimizer='sgd', loss='categorical_crossentropy')

y_train_c = to_categorical(y_train)
model1.fit(x_train, y_train_c, batch_size=32, epochs=10)

Epoch 1/10


ValueError: Input 0 of layer "conv2d_4" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 3072)

In [53]:
pred = model1.predict(x_test)
y_pred1 = list(map(str, np.argmax(pred, axis=-1)))

score1 = accuracy_score(y_test, y_pred1)
print(f"{score = }")

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
score = 0.42473333333333335
